### LLM 장착한 langgraph

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI # llm 모델 사용하기 위한 모듈
from langchain_core.messages import HumanMessage, AIMessage # message 양식 처리를 위한 모듈

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# 1. state 정의, 모델 정의
# 2. 노드 만들기
# 3. 그래프 생성
# 4. 실행해보기

### 1. State 정의, 모델 정의

In [2]:
class ChatState(TypedDict):
    user_input : str    # 처음 입력한 질문
    llm_response : str  # AI 최종 답변
    conversation_history : list # 그 동안의 대화 내용 저장하기

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.2
)

### 2. 노드 만들기
1. 사람이 질문 일렵
2. llm이 답변

In [4]:
def get_user_input(state : ChatState) -> ChatState: # 확인용
    user_message = state.get("user_input", "")
    print("처음 입력한 메시지", user_message)
    print("="*50)
    return state

def call_llm(state : ChatState) -> ChatState :
    user_message = state.get("user_input", "")

    # llm 호출
    response = llm.invoke([HumanMessage(content=user_message)])
    print("response" ,  response)
    print("-"*50)
    print("히스토리", state.get("conversation_history", []))
    
    print("response 내용", state.get("response", ""))   # 아직 state에 response 값을 업데이트 하지 않은 상태
    print("="*50)

    return {
        "user_input" : user_message,
        "llm_response" : response,
        "conversation-history" : state.get("conversation_history", [])
    }


### 그래프 생성

In [ ]:
workflow = StateGraph(ChatState)

workflow.add_node("input", get_user_input)
workflow.add_node("llm", call_llm)

workflow.add_edge("input", "llm")
workflow.set_entry_point("input")
workflow.set_finish_point("llm")
app = workflow.compile()
app

### 과제
1. llm_response 업데이트 되었는지 확인하는 노드 추가
2. history 내용을 업데이트 하도록 노드 추가
3. history 내용 확인하는 노드도 추가해보세요

In [ ]:
# 히스토리 내용이 업데이트(힌트이자 정답일지도)
def history_add(state: ChatState) -> ChatState:
    history = state.get("conversation_history", [])
    history.append({    # 일단 append 로 넣어보자, 나중엔 add_message 로 처리
        "user": state["user_input"],
        "ai": state["llm_response"]
    })
    return {
        "user_input" : state["user_input"],
        "llm_response" : state["llm_response"],
        "conversation_history" : history
    }